In [20]:
from pymongo import MongoClient
import pprint
import warnings
import datetime

warnings.filterwarnings('ignore')

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.bdlp_db_def
collection = db.bdlp_c_def

In [3]:
#Listado de todas las publicaciones de un autor determinado.
author="Sanjeev Saxena"
print("------Listado de todas las publicaciones de", author,": ------")
cursor = collection.find({"Authors":author}, {"title": 1, '_id': 0})

i = 0
for record in cursor:
    print(i, ":", record["title"])
    i+=1

------Listado de todas las publicaciones de Sanjeev Saxena : ------
0 : Parallel Integer Sorting and Simulation Amongst CRCW Models.
1 : Optimal Sublogarithmic Time Parallel Algorithms on Rooted Forests.
2 : Parallel Algorithms for Finding the Most Vital Edge in Weighted Graphs.
3 : An efficient parallel algorithm for building the separating tree.
4 : Fast parallel edge colouring of graphs.
5 : Faster algorithm to find anti-risk path between two nodes of an undirected graph.
6 : Maximum cardinality neighbourly sets in quadrilateral free graphs.
7 : Maximal independent sets in a generalisation of caterpillar graph.
8 : Algorithms for testing occurrences of length 4 patterns in permutations.
9 : A still simpler way of introducing interior-point method for linear programming.
10 : Efficient VLSI Parallel Algorithm for Delaunay Triangulation on Orthogonal Tree Network in Two and Three Dimensions.
11 : Two-Coloring Linked Lists is NC^1-Complete for Logarithmic Space.
12 : Dominance made sim

In [4]:
#Número de publicaciones de un autor determinado.
author="Sanjeev Saxena"
print("------Numero de publicaciones de",author,": ------")
cursor = collection.count({"Authors":author})
print (cursor)

------Numero de publicaciones de Sanjeev Saxena : ------
44


In [5]:
#Número de artículos en revista para el año 2017

pub_type = "article"
year = 2017
print("------Numero de",pub_type,"en revista para el año",year,": ------")
cursor = collection.count({"type":pub_type,"year":year})
print (cursor)


------Numero de article en revista para el año 2017 : ------
150622


In [6]:
#Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total.
print("------Numero de autores ocasionales : ------")
        
cursor = collection.aggregate([{'$unwind': "$Authors" }, 
                             {'$sortByCount': "$Authors"}, 
                             {'$match': {'count':{'$lt':5} } }, 
                             {'$count': "autores ocasionales"} ], 
                            allowDiskUse=True)

for record in cursor:
    print("El numero de autores ocasionales es",record["autores ocasionales"])


------Numero de autores ocasionales : ------
El numero de autores ocasionales es 1767280


In [7]:
#Número de artículos de revista (article) y número de artículos en congresos
#(inproceedings) de los diez autores con más publicaciones totales.

print("------diez autores con más publicaciones totales de articulos y revistas : ------")
        
cursor = collection.aggregate([{'$match': { '$or': [ {'type':"inproceedings"},{'type':"article"}]}},
                              {'$unwind': "$Authors" },
                              {'$group': {'_id':{ 'Authors': "$Authors"},'Publicaciones': {'$sum':1}}}, 
                              {'$sort': {"Publicaciones":-1}}, 
                              {'$limit':11}],
                              allowDiskUse=True)

i = 0   
for record in cursor:
    print(i, ":",record["_id"]["Authors"],"con",record['Publicaciones'], "publicaciones")
    i+=1

------diez autores con más publicaciones totales de articulos y revistas : ------
0 : None con 5840 publicaciones
1 : H. Vincent Poor con 1672 publicaciones
2 : Mohamed-Slim Alouini con 1293 publicaciones
3 : Philip S. Yu con 1224 publicaciones
4 : Wei Wang con 1194 publicaciones
5 : Wei Zhang con 1170 publicaciones
6 : Lajos Hanzo con 1164 publicaciones
7 : Wen Gao 0001 con 1160 publicaciones
8 : Wei Li con 1149 publicaciones
9 : Yang Liu con 1092 publicaciones
10 : Yu Zhang con 1084 publicaciones


In [8]:
#Número medio de autores de todas las publicaciones que tenga en su conjunto de datos.
print("------Numero medio de autores de todas las publicaciones que tenga en su conjunto de datos : ------")
        
cursor = collection.aggregate([{'$unwind': "$Authors" }, 
                             {'$group': {'_id': '$_id', 'count':{'$sum':1}}},
                             {'$group': {'_id':'null', 'avg':{'$avg':'$count'}}}],
                             allowDiskUse=True)

for record in cursor:
    print(record["avg"])

------Numero medio de autores de todas las publicaciones que tenga en su conjunto de datos : ------
3.0131759743876616


In [9]:
#Listado de coautores de un autor
author="Sanjeev Saxena"
print("------Listado de coautores de un autor : ------")

cursor = collection.aggregate([{"$match":{"Authors":author}},
                             {"$unwind": '$Authors'},
                             {'$group': {'_id':{ 'Authors': "$Authors"}}},
                             {'$match':{'_id':{'$ne':author}}}],
                              allowDiskUse=True)

for record in cursor:
    print(record["_id"]["Authors"])

------Listado de coautores de un autor : ------
K. Nandan Babu
Ridhi Garg
Richa Shreyam
K. Jeevan Madhu
V. C. Prasad
Neethi K. S.
Sanjeev Saxena
Kurt Mehlhorn
P. C. P. Bhatt
Xiaojun Shen
G. Sajith
S. Das
Jay Mahadeokar
Sudarshan Banerjee
Shailza Gotra
Yijie Han
N. Malahal Rao
K. V. R. C. N. Kishore
Krzysztof Diks
Anamika Banwari
Torben Hagerup
Tomasz Radzik
Pramod Chandra P. Bhatt
V. Yugandhar
Anjeneya Swami Kare
Niloy Chakrabarti
Sagar Kalra
M. R. Tripathy


In [10]:
#Edad de los 5 autores con un periodo de publicaciones más largo
print("------Edad de los 5 autores con un periodo de publicaciones mas largo : ------")

cursor = collection.aggregate([{"$unwind": '$Authors'},
                              {'$match':{'year':{'$ne':"None"}}},
                              {'$group': {'_id': "$Authors",'year': {'$push': '$year'}}}, 
                              {'$project': {'max': {'$max': "$year"},'min':{ '$min': "$year"}}}, 
                              {'$project': {'edad': {'$subtract': ["$max","$min"]}}},
                              {'$sort': {"edad":-1}}, 
                              {'$limit':5}], 
                              allowDiskUse=True)

for record in cursor:
    print(record["_id"]," tiene edad",record["edad"])

------Edad de los 5 autores con un periodo de publicaciones mas largo : ------
Alan M. Turing  tiene edad 75
Rudolf Carnap  tiene edad 71
David Nelson  tiene edad 69
Martin Davis  tiene edad 65
Eric Weiss  tiene edad 64


In [11]:
#Número de autores novatos, es decir, que tengan una Edad menor de 5 años.
print("------Numero de autores novatos, es decir, que tengan una Edad menor de 5 años. : ------")

cursor = collection.aggregate([{"$unwind": '$Authors'},
                              {'$match':{'year':{'$ne':"None"}}},
                              {'$group': {'_id': "$Authors",'year': {'$push': '$year'}}}, 
                              {'$project': {'max': {'$max': "$year"},'min':{ '$min': "$year"}}}, 
                              {'$project': {'edad': {'$subtract': ["$max","$min"]}}},
                              {'$match':{'edad':{'$lt':5}}},
                              {'$count': "autores novatos"} ], 
                              allowDiskUse=True)
for record in cursor:
    print("El numero de autores novatos es:",record["autores novatos"])

------Numero de autores novatos, es decir, que tengan una Edad menor de 5 años. : ------
El numero de autores novatos es: 1680262


In [12]:
#Porcentaje de publicaciones en revistas con respecto al total de publicaciones.
print("------Porcentaje de publicaciones en revistas con respecto al total de publicaciones : ------")

cursor = collection.aggregate([{'$match': { 'type': "article"} }, 
                               {'$group': {'_id':'$type', 'count':{'$sum':1}}}, 
                               {'$project':{'_id':0,'por_articles':{ '$multiply': [{'$divide': [100, collection.count()]}, "$count"]}}}],
                               allowDiskUse = True)
for record in cursor:
    print("El porcentaje de articulos de revista es",record["por_articles"])

------Porcentaje de publicaciones en revistas con respecto al total de publicaciones : ------
El porcentaje de articulos de revista es 45.36040528233151


In [ ]:
#INDICES (EVALUACION DE TIEMPOS)

In [ ]:
#TIEMPOS SIN INDICES 

In [23]:
#Listado de todas las publicaciones de un autor determinado.
author="Sanjeev Saxena"
print("------Listado de todas las publicaciones de", author,": ------")
a = datetime.datetime.now()
cursor = collection.find({"Authors":author}, {"title": 1, '_id': 0})

b = datetime.datetime.now()
c = b - a
c.microseconds


------Listado de todas las publicaciones de Sanjeev Saxena : ------


163

In [24]:
#Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total.
print("------Numero de autores ocasionales : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{'$unwind': "$Authors" }, 
                             {'$sortByCount': "$Authors"}, 
                             {'$match': {'count':{'$lt':5} } }, 
                             {'$count': "autores ocasionales"} ], 
                            allowDiskUse=True)
b = datetime.datetime.now()
c = b - a
c.microseconds


------Numero de autores ocasionales : ------


480090

In [25]:
#Número de artículos de revista (article) y número de artículos en congresos
#(inproceedings) de los diez autores con más publicaciones totales.

print("------diez autores con más publicaciones totales de articulos y revistas : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{'$match': { '$or': [ {'type':"inproceedings"},{'type':"article"}]}},
                              {'$unwind': "$Authors" },
                              {'$group': {'_id':{ 'Authors': "$Authors"},'Publicaciones': {'$sum':1}}}, 
                              {'$sort': {"Publicaciones":-1}}, 
                              {'$limit':11}],
                              allowDiskUse=True)

b = datetime.datetime.now()
c = b - a
c.microseconds

------diez autores con más publicaciones totales de articulos y revistas : ------


681373

In [26]:
#Edad de los 5 autores con un periodo de publicaciones más largo
print("------Edad de los 5 autores con un periodo de publicaciones mas largo : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{"$unwind": '$Authors'},
                              {'$match':{'year':{'$ne':"None"}}},
                              {'$group': {'_id': "$Authors",'year': {'$push': '$year'}}}, 
                              {'$project': {'max': {'$max': "$year"},'min':{ '$min': "$year"}}}, 
                              {'$project': {'edad': {'$subtract': ["$max","$min"]}}},
                              {'$sort': {"edad":-1}}, 
                              {'$limit':5}], 
                              allowDiskUse=True)

b = datetime.datetime.now()
c = b - a
c.microseconds

------Edad de los 5 autores con un periodo de publicaciones mas largo : ------


666450

In [ ]:
#Creación de indice 
#db.dblp.createIndex({"author":1})

In [31]:
#Listado de todas las publicaciones de un autor determinado.
author="Sanjeev Saxena"
print("------Listado de todas las publicaciones de", author,": ------")
a = datetime.datetime.now()
cursor = collection.find({"Authors":author}, {"title": 1, '_id': 0})

b = datetime.datetime.now()
c = b - a
c.microseconds

------Listado de todas las publicaciones de Sanjeev Saxena : ------


120

In [33]:
#Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total.
print("------Numero de autores ocasionales : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{'$unwind': "$Authors" }, 
                             {'$sortByCount': "$Authors"}, 
                             {'$match': {'count':{'$lt':5} } }, 
                             {'$count': "autores ocasionales"} ], 
                            allowDiskUse=True)
b = datetime.datetime.now()
c = b - a
c.microseconds


------Numero de autores ocasionales : ------


79296

In [36]:
#Número de artículos de revista (article) y número de artículos en congresos
#(inproceedings) de los diez autores con más publicaciones totales.

print("------diez autores con más publicaciones totales de articulos y revistas : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{'$match': { '$or': [ {'type':"inproceedings"},{'type':"article"}]}},
                              {'$unwind': "$Authors" },
                              {'$group': {'_id':{ 'Authors': "$Authors"},'Publicaciones': {'$sum':1}}}, 
                              {'$sort': {"Publicaciones":-1}}, 
                              {'$limit':11}],
                              allowDiskUse=True)

b = datetime.datetime.now()
c = b - a
c.microseconds

------diez autores con más publicaciones totales de articulos y revistas : ------


542628

In [41]:
#Edad de los 5 autores con un periodo de publicaciones más largo
print("------Edad de los 5 autores con un periodo de publicaciones mas largo : ------")

a = datetime.datetime.now()

cursor = collection.aggregate([{"$unwind": '$Authors'},
                              {'$match':{'year':{'$ne':"None"}}},
                              {'$group': {'_id': "$Authors",'year': {'$push': '$year'}}}, 
                              {'$project': {'max': {'$max': "$year"},'min':{ '$min': "$year"}}}, 
                              {'$project': {'edad': {'$subtract': ["$max","$min"]}}},
                              {'$sort': {"edad":-1}}, 
                              {'$limit':5}], 
                              allowDiskUse=True)

b = datetime.datetime.now()
c = b - a
c.microseconds

------Edad de los 5 autores con un periodo de publicaciones mas largo : ------


171503